In [1]:
from itertools import chain
from random import random

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

from society.action import Action
from society.agent import Agent, TrainableAgent
from society.simulations.weighted import WeightedNetworkSimulation
from society.strategies.gameplay.constant import AllC, AllD
from society.strategies.gameplay.qlearning import TabularQLearningGameplayStrategy
from society.strategies.gameplay.random import RandomGameplayStrategy
from society.strategies.gameplay.tft import TitForTat
from society.visualisation.network import *

plt.rcParams["figure.figsize"] = (10, 6)

In [2]:
def generate_population(communities, size):
    population = communities * size

    agents = [
        Agent(TabularQLearningGameplayStrategy(lookback=2, epsilon=0.2), i, population)
        for i in range(population)
    ]

    G = nx.connected_caveman_graph(communities, size)

    weights_matrix = np.zeros((population, population))
    for u, v, d in G.edges(data=True):
        weights_matrix[u, v] = weights_matrix[v, u] = 1.0
        try:
            d["weight"] = weights_matrix[u, v]
        except:
            d["weight"] = 0

    return agents, weights_matrix, G

In [3]:
def compute_policies(agents):
    policies = [
        agent.gameplay_strategy._q_table.argmax(axis=-1)
        for agent in agents
    ]

    return [
        "".join(
            [
                ("C", "D")[policy[i, j]]  # , k
                for i in range(4)
                for j in range(4)
                # for k in range(4)
            ]
        )
        for policy in policies
    ]

def calculate_cooperativeness(history):
    count = history.count(Action.COOPERATE)

    return count / len(history)

In [4]:
# PAYOFF_LABELS = ["(C, C)", "(C, D)", "(D, C)", "(D, D)"]

PAYOFF_LABELS = ["R", "S", "T", "P"]

POPULATION = (2, 32)
ROUNDS = 8_000

run_rewards = []
run_cumulative_rewards = []
run_action_histories = []
run_reward_histories = []
run_policies = []

for run in range(20):
    # Generate a new population
    agents, weights_matrix, G = generate_population(*POPULATION)

    # Run a number of rounds
    sim = WeightedNetworkSimulation(agents, weights_matrix)

    # Training phase
    sim.reset()
    for i in tqdm(range(ROUNDS // 2), desc=f"Training Run {run + 1}"):
        sim.play_round(train=True)

    # Evaluation phase
    for agent in agents:
        agent.gameplay_strategy._epsilon = 0.0

    sim.reset()
    for i in tqdm(range(ROUNDS // 2), desc=f"Evaluation Run {run + 1}"):
        sim.play_round(train=False)

    # Store results
    run_rewards.append(sim.rewards)
    run_action_histories.append(sim.action_histories)
    run_reward_histories.append(sim.reward_histories)
    run_policies.append(compute_policies(agents))

    # Output statistics for the best and worst agents
    cumulative_rewards = [(i, sum(history)) for i, history in enumerate(sim.reward_histories)]
    cumulative_rewards.sort(key=lambda x: x[1], reverse=True)

    tqdm.write(
        f"BEST AGENT: {cumulative_rewards[0][1]} ({calculate_cooperativeness(list(chain(*sim.action_histories[cumulative_rewards[0][0]])))}, {run_policies[-1][cumulative_rewards[0][0]]})"
    )
    tqdm.write(
        f"WORST AGENT: {cumulative_rewards[-1][1]} ({calculate_cooperativeness(list(chain(*sim.action_histories[cumulative_rewards[-1][0]])))}, {run_policies[-1][cumulative_rewards[-1][0]]})"
    )


Training Run 1:   0%|          | 0/4000 [00:00<?, ?it/s]

Evaluation Run 1:   0%|          | 0/4000 [00:00<?, ?it/s]

BEST AGENT: 33938 (0.10903387703889586, DDDCCCDCCDDCDDDD)
WORST AGENT: 17001 (1.0, CCCDCCCCCCCCCCCC)


Training Run 2:   0%|          | 0/4000 [00:00<?, ?it/s]

Evaluation Run 2:   0%|          | 0/4000 [00:00<?, ?it/s]

...

Evaluation Run 19:   0%|          | 0/4000 [00:00<?, ?it/s]

BEST AGENT: 32735 (0.14989293361884368, DCDCDDCCDCDCDCDC)
WORST AGENT: 18927 (1.0, CCCCCCCCCCCDCCCC)


Training Run 20:   0%|          | 0/4000 [00:00<?, ?it/s]

Evaluation Run 20:   0%|          | 0/4000 [00:00<?, ?it/s]

BEST AGENT: 33545 (0.1378585558852621, DCDCDDCDDCDDDCCC)
WORST AGENT: 18873 (1.0, CCCCCCCCCCCCCCCD)


In [7]:
cumulative_reward_policy_ranks = {}

for policies, reward_histories in zip(run_policies, run_reward_histories):
    cumulative_rewards = [(i, sum(history)) for i, history in enumerate(reward_histories)]
    cumulative_rewards.sort(key=lambda x: x[1], reverse=True)

    for rank, (i, cumulative_reward) in enumerate(cumulative_rewards):
        policy = policies[i]

        if policy not in cumulative_reward_policy_ranks:
            cumulative_reward_policy_ranks[policy] = []

        cumulative_reward_policy_ranks[policy].append(rank + 1)

mean_cumulative_reward_policy_ranks = {policy: np.mean(ranks) for policy, ranks in cumulative_reward_policy_ranks.items()}

for policy in sorted(mean_cumulative_reward_policy_ranks, key=lambda x: mean_cumulative_reward_policy_ranks[x]):
    print(f"{mean_cumulative_reward_policy_ranks[policy]:<24} {policy}")

1.0                      DDDCCCDCCDDCDDDD
1.0                      DCDCDCCCDDDCDCCC
1.0                      DCDDCDCCCCCDCCCC
1.0                      DCDCDDCCCCDCCCCC
1.0                      DCCDCCCCDCCCCCDC
1.0                      DCDDCCDCCDDDCDDD
1.0                      DDDCCCCDDCDCDCCC
1.0                      DCDCCDDCCCDCDDDC
1.0                      DCDCCCDCDDDCCCDD
1.0                      DCDCDCDCDCDCCCDC
1.0                      DCDDCCCCDCCCCCCC
1.0                      DCDDCCCDCCDDCCDD
1.0                      DDDCCCDCCCDDDCDD
1.0                      DCDCCCCCDCDCDCCC
1.0                      DCDDCCCCDCDDCDDD
1.0                      DCCDDCCDDDCDCCDC
1.0                      DDDCCCCCCCDCCDCD
1.0                      DCDCCDCCCCDCDCDC
1.0                      DCDCDDCCDCDCDCDC
1.0                      DCDCDDCDDCDDDCCC
2.0                      DCDCCCDDCCDDCCDD
2.0                      DCDDCDCDCCDCCCDC
2.0                      DCDDDCDDCCDCCCCC
2.0                      DCDCDDCCD